In [1]:
# ! pip install statsmodels

In [2]:
import pandas as pd
import numpy as np

In [3]:
card_df = pd.read_csv('../vip_card_data_merged.csv')
card2_df = pd.read_csv('../vipcard3.csv')
card3_df = pd.read_csv('../price.csv')

In [4]:
merged_df = pd.merge(card_df, card2_df, on='발급회원번호', how='inner')

In [5]:
data = pd.merge(merged_df,card3_df, on='발급회원번호',how='inner')

In [6]:
data.to_csv('../컬럼확인용.csv', index =False)

In [7]:
def preprocessing_data(data):
    data['이용건수_B0M'] = data[['이용건수_일시불_B0M','이용건수_체크_B0M','이용건수_할부_B0M']].sum(axis=1)
    data['이용금액_B0M'] = data[['이용금액_일시불_B0M','이용금액_체크_B0M','이용금액_할부_B0M']].sum(axis=1)
    data['최종이용일자'] = data[['최종이용일자_일시불','최종이용일자_체크','최종이용일자_할부']].max(axis=1)
    data['이용후경과월'] = data[['이용후경과월_일시불','이용후경과월_체크','이용후경과월_할부']].sum(axis=1)
    data['이용건수_R12M'] = data[['이용건수_일시불_R12M','이용건수_체크_R12M','이용건수_할부_R12M']].sum(axis=1)
    data['이용금액_R12M'] = data[['이용금액_일시불_R12M','이용금액_체크_R12M','이용금액_할부_R12M']].sum(axis=1)
    data['최대이용금액_R12M'] = data[['최대이용금액_일시불_R12M','최대이용금액_체크_R12M','최대이용금액_할부_R12M']].sum(axis=1)
    data['이용개월수_R12M'] = data[['이용개월수_일시불_R12M','이용개월수_체크_R12M','이용개월수_할부_R12M']].sum(axis=1)
    data['이용건수_R6M'] = data[['이용건수_일시불_R6M','이용건수_체크_R6M','이용건수_할부_R6M']].sum(axis=1)
    data['이용금액_R6M'] = data[['이용금액_일시불_R6M','이용금액_체크_R6M','이용금액_할부_R6M']].sum(axis=1)
    data['이용개월수_R6M'] = data[['이용개월수_일시불_R6M','이용개월수_체크_R6M','이용개월수_할부_R6M']].sum(axis=1)
    data['이용건수_R3M'] = data[['이용건수_일시불_R3M','이용건수_체크_R3M','이용건수_할부_R3M']].sum(axis=1)
    data['이용금액_R3M'] = data[['이용금액_일시불_R3M','이용금액_체크_R3M','이용금액_할부_R3M']].sum(axis=1)
    data['이용개월수_R3M'] = data[['이용개월수_일시불_R3M','이용개월수_체크_R3M','이용개월수_할부_R3M']].sum(axis=1)
    data['회원여부_이용가능'] = data[['회원여부_이용가능','회원여부_이용가능_CA','회원여부_이용가능_카드론']].sum(axis=1)
    data['유효카드수'] = data[['유효카드수_신용체크','유효카드수_신용','유효카드수_체크']].sum(axis=1)
    data['이용금액_증감률'] = (data['이용금액_R3M'] - data['이용금액_R6M']) / (data['이용금액_R6M'] + 1)
    data['최근_이용금액_비중'] = data['이용금액_B0M'] / (data['이용금액_R12M'] + 1)
    # data['활동성_R12M'] = data['이용금액_R12M'] / (data['이용개월수_R12M'] + 1)
    # data['활동성_R3M'] = data['이용금액_R3M'] / (data['이용개월수_R3M'] + 1)
    data['VIP등급코드'] = data['VIP등급코드'].str.replace("_","0").astype(int)
    data['이용금액대'] = data['이용금액대'].str.extract(r'^(\d+)').astype(int)
    data['연령'] = data['연령'].str.replace("대|이상","",regex=True).astype(int)
    cols_to_drop = [
    # B0M
    '이용건수_일시불_B0M', '이용건수_체크_B0M', '이용건수_할부_B0M',
    '이용금액_일시불_B0M', '이용금액_체크_B0M', '이용금액_할부_B0M',
    
    # 최종 이용일자
    '최종이용일자_일시불', '최종이용일자_체크', '최종이용일자_할부',
    '이용후경과월_일시불', '이용후경과월_체크', '이용후경과월_할부',
    
    # R12M
    '이용건수_일시불_R12M', '이용건수_체크_R12M', '이용건수_할부_R12M',
    '이용금액_일시불_R12M', '이용금액_체크_R12M', '이용금액_할부_R12M',
    '최대이용금액_일시불_R12M', '최대이용금액_체크_R12M', '최대이용금액_할부_R12M',
    '이용개월수_일시불_R12M', '이용개월수_체크_R12M', '이용개월수_할부_R12M',
    
    # R6M
    '이용건수_일시불_R6M', '이용건수_체크_R6M', '이용건수_할부_R6M',
    '이용금액_일시불_R6M', '이용금액_체크_R6M', '이용금액_할부_R6M',
    '이용개월수_일시불_R6M', '이용개월수_체크_R6M', '이용개월수_할부_R6M',

    # R3M
    '이용건수_일시불_R3M', '이용건수_체크_R3M', '이용건수_할부_R3M',
    '이용금액_일시불_R3M', '이용금액_체크_R3M', '이용금액_할부_R3M',
    '이용개월수_일시불_R3M', '이용개월수_체크_R3M', '이용개월수_할부_R3M',

    # 회원여부
    '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론',

    # 유효카드수
    '유효카드수_신용체크', '유효카드수_신용', '유효카드수_체크',
    #발급회원번호 제거
    '발급회원번호',
    # 파생변수 생성컬럼 제거
    

    ]

    data.drop(columns=cols_to_drop, inplace=True)



In [8]:
preprocessing_data(data)

In [9]:
data

,이용금액_업종기준,VIP등급코드,남녀구분코드,연령,입회경과개월수_신용,탈회횟수_누적,최종탈회후경과월,탈회횟수_발급6개월이내,탈회횟수_발급1년이내,마케팅동의여부,...,이용개월수_R12M,이용건수_R6M,이용금액_R6M,이용개월수_R6M,이용건수_R3M,이용금액_R3M,이용개월수_R3M,유효카드수,이용금액_증감률,최근_이용금액_비중
0,0,0,2,30,42,1,44,0,0,0,...,4,0,246480,2,0,-84301,0,4,-1.342014,0.000000
1,200495,0,2,30,2,0,0,0,0,0,...,12,18,-37155,6,16,139165,3,2,-4.745653,0.052493
2,0,0,2,40,180,0,0,0,0,1,...,0,-1,0,0,0,-66677,0,6,-66677.000000,-0.000000
3,2720324,0,1,50,45,2,45,0,0,1,...,12,103,5502584,7,52,2260947,3,2,-0.589112,0.085102
4,1561973,0,1,40,44,1,100,0,0,0,...,15,194,3444713,7,87,1097135,3,8,-0.681502,0.078296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,3263556,6,2,40,7,0,0,0,0,1,...,31,405,10838430,15,206,5004667,8,8,-0.538248,0.066639
2999996,28531,0,1,50,129,0,0,0,0,1,...,17,183,3314410,8,92,1893145,5,6,-0.428814,0.091482
2999997,815392,0,2,30,35,0,0,0,0,1,...,23,347,8051723,12,183,4381632,6,8,-0.455814,0.139295
2999998,62299,7,2,40,28,1,43,0,0,1,...,25,380,8243196,11,172,3247711,4,10,-0.606013,0.101726


In [10]:
data.to_csv("../내돈내산.csv",index=False)